# Create JAMS
This notebook creates the jams audio annotations which will be used in the training.
https://github.com/marl/jams

In [1]:
import jams
import os
import sox
from glob import glob
from multiprocessing import Process
from joblib import Parallel, delayed

from tqdm import tqdm
import pandas as pd

In [2]:
jams.__version__

'0.3.4'

In [3]:
# get the unique class names of the birds in our training set
df = pd.read_csv('data/train.csv')
classes = df['ebird_code'].unique()

classes

array(['aldfly', 'ameavo', 'amebit', 'amecro', 'amegfi', 'amekes',
       'amepip', 'amered', 'amerob', 'amewig', 'amewoo', 'amtspa',
       'annhum', 'astfly', 'baisan', 'baleag', 'balori', 'banswa',
       'barswa', 'bawwar', 'belkin1', 'belspa2', 'bewwre', 'bkbcuc',
       'bkbmag1', 'bkbwar', 'bkcchi', 'bkchum', 'bkhgro', 'bkpwar',
       'bktspa', 'blkpho', 'blugrb1', 'blujay', 'bnhcow', 'boboli',
       'bongul', 'brdowl', 'brebla', 'brespa', 'brncre', 'brnthr',
       'brthum', 'brwhaw', 'btbwar', 'btnwar', 'btywar', 'buffle',
       'buggna', 'buhvir', 'bulori', 'bushti', 'buwtea', 'buwwar',
       'cacwre', 'calgul', 'calqua', 'camwar', 'cangoo', 'canwar',
       'canwre', 'carwre', 'casfin', 'caster1', 'casvir', 'cedwax',
       'chispa', 'chiswi', 'chswar', 'chukar', 'clanut', 'cliswa',
       'comgol', 'comgra', 'comloo', 'commer', 'comnig', 'comrav',
       'comred', 'comter', 'comyel', 'coohaw', 'coshum', 'cowscj1',
       'daejun', 'doccor', 'dowwoo', 'dusfly', 'eargre',

In [4]:
def create_jam(audiofile, weakdf, verbose=False):
    
    # setup paths from args
    base_folder = 'data'
    #split_folder = os.path.join(base_folder, split)
    
    # create jam
    jam=jams.JAMS()

    # create annotation
    ann = jams.Annotation('tag_open')
    duration = 10.0  # all files have been reduced to 10 seconds in length
    ann.duration = duration
    
    # get file label
    audiobase = os.path.basename(audiofile)
    fid = os.path.splitext(audiobase)[0]
    
    if verbose:
        print('audiobase', audiobase)
        print('fid', fid)
    
    # add tag for each label
    # right now these all have single tags but we can pull the also tags
    # from the api later if MVP works
    
    labels = weakdf[weakdf['filename'] == fid + '.mp3'].ebird_code.values
    assert len(labels) > 0
    if verbose:
        print('labels', labels)
        
    # Add tag for each label
    for label in labels:
        ann.append(time=0, duration=duration, value=label, confidence=1)

    # fill jam metadata
    jam.file_metadata.title = audiobase
    jam.file_metadata.release = '1.0'
    jam.file_metadata.duration = duration
    jam.file_metadata.artist = ''
    
    # fill annotation metadata
    ann.annotation_metadata.version = '1.0'
    ann.annotation_metadata.corpus = 'Cornell birdsong-recognition'
    ann.annotation_metadata.data_source = 'Kaggle'
    
    # add annotation to jam
    jam.annotations.append(ann)
    
    jam.save(audiofile.replace('.wav', '.jams'))
    
    return jam

In [5]:
# test 
jam = create_jam(audiofile='data/train_10sec/XC1634.wav', weakdf=df, verbose=True)
print(jam)

audiobase XC1634.wav
fid XC1634
labels ['houwre']
{
  "annotations": [
    {
      "annotation_metadata": {
        "curator": {
          "name": "",
          "email": ""
        },
        "annotator": {},
        "version": "1.0",
        "corpus": "Cornell birdsong-recognition",
        "annotation_tools": "",
        "annotation_rules": "",
        "validation": "",
        "data_source": "Kaggle"
      },
      "namespace": "tag_open",
      "data": [
        {
          "time": 0.0,
          "duration": 10.0,
          "value": "houwre",
          "confidence": 1
        }
      ],
      "sandbox": {},
      "time": 0,
      "duration": 10.0
    }
  ],
  "file_metadata": {
    "title": "XC1634.wav",
    "artist": "",
    "release": "1.0",
    "duration": 10.0,
    "identifiers": {},
    "jams_version": "0.3.4"
  },
  "sandbox": {}
}


## Create all JAMS files

In [6]:
base_folder = 'data/train_10sec'
weakdf = pd.read_csv('data/train.csv')

audiofiles = glob(os.path.join(base_folder, '*.wav'))

# create the jams files
jams = Parallel(n_jobs=4, verbose=1)(delayed(create_jam)(i, weakdf) for i in audiofiles)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    8.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   25.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   53.7s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  2.5min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  5.2min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed:  7.3min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed:  9.5min
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed: 11.8min
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed: 14.6min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed: 17.2min
[Parallel(n_jobs=4)]: Done 8442 tasks      | elapsed: 21.7min
[Parallel(n_jobs=4)]: Done 9792 tasks      | elapsed: 27.3min
[Parallel(n_jobs=4)]: Done 11242 tasks      | elapsed: 32.8mi